In [1]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV

In [2]:
import pandas as pd

In [3]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [4]:
data = pd.read_csv('train.csv')

In [7]:
data = data.drop(['PassengerId', 'Cabin', 'Name', 'Ticket'], axis=1)

In [8]:
np.random.seed(0)
idx = np.random.rand(len(data))
train_data = data.iloc[idx > 0.1].reset_index(drop=True)
test_data = data.iloc[idx <= 0.1].reset_index(drop=True)

In [9]:
train_data[['Survived', 'Pclass']].groupby('Pclass').mean()

Survived
Pclass          
1       0.623037
2       0.469512
3       0.238839

In [10]:
train_data[['Survived', 'Sex']].groupby('Sex').mean()

Survived
Sex             
female  0.734767
male    0.187023

In [11]:
train_data[['Survived', 'Pclass', 'Sex']].groupby(['Pclass', 'Sex']).mean()

Survived
Pclass Sex             
1      female  0.964286
       male    0.355140
2      female  0.924242
       male    0.163265
3      female  0.488372
       male    0.137931

In [12]:
def predict_by_pclass(data):
    pred = pd.DataFrame(np.zeros(len(data)), columns=['Pred'])
    pred[data['Pclass']==1] = 0.6230
    pred[data['Pclass']==2] = 0.4695
    pred[data['Pclass']==3] = 0.2388
    return pred

In [13]:
pred = predict_by_pclass(test_data)
acc = accuracy_score(test_data['Survived'], pred > 0.5)
auc = roc_auc_score(test_data['Survived'], pred)
print('Accuracy = %.4f, AUC = %.4f' % (acc, auc))

Accuracy = 0.6591, AUC = 0.6860


In [14]:
def predict_by_sex(data):
    pred = pd.DataFrame(np.zeros(len(data)), columns=['Pred'])
    pred[data['Sex']=='female'] = 0.7348
    pred[data['Sex']=='male'] = 0.1870
    return pred

In [15]:
pred = predict_by_sex(test_data)
acc = accuracy_score(test_data['Survived'], pred > 0.5)
auc = roc_auc_score(test_data['Survived'], pred)
print('Accuracy = %.4f, AUC = %.4f' % (acc, auc))

Accuracy = 0.7955, AUC = 0.7875


In [16]:
def predict_by_pclass_sex(data):
    pred = pd.DataFrame(np.zeros(len(data)), columns=['Pred'])
    pred[(data['Pclass']==1)&(data['Sex']=='female')] = 0.9643
    pred[(data['Pclass']==1)&(data['Sex']=='male')] = 0.3551
    pred[(data['Pclass']==2)&(data['Sex']=='female')] = 0.9242
    pred[(data['Pclass']==2)&(data['Sex']=='male')] = 0.1633
    pred[(data['Pclass']==3)&(data['Sex']=='female')] = 0.4883
    pred[(data['Pclass']==3)&(data['Sex']=='male')] = 0.1379
    return pred

In [17]:
pred = predict_by_pclass_sex(test_data)
acc = accuracy_score(test_data['Survived'], pred > 0.5)
auc = roc_auc_score(test_data['Survived'], pred)
print('Accuracy = %.4f, AUC = %.4f' % (acc, auc))

Accuracy = 0.7614, AUC = 0.8710


In [18]:
mean_age = train_data['Age'].mean()
train_data['Age'] = train_data['Age'].fillna(value=mean_age)
test_data['Age'] = test_data['Age'].fillna(value=mean_age)

In [19]:
min_age, max_age = train_data['Age'].min(), train_data['Age'].max()
bins = np.linspace(min_age, max_age, 5)
labels = [1, 2, 3, 4]
train_data['Age_discrete'] = pd.cut(train_data['Age'], bins=bins, labels=labels, include_lowest=True)
test_data['Age_discrete'] = pd.cut(test_data['Age'], bins=bins, labels=labels, include_lowest=True)

In [20]:
train_data[['Survived', 'Age_discrete']].groupby('Age_discrete').mean()

Survived
Age_discrete          
1             0.512195
2             0.345850
3             0.397163
4             0.272727

In [21]:
def predict_by_age(data):
    pred = pd.DataFrame(np.zeros(len(data)), columns=['Pred'])
    pred[data['Age_discrete']==1] = 0.5122
    pred[data['Age_discrete']==2] = 0.3458
    pred[data['Age_discrete']==3] = 0.3972
    pred[data['Age_discrete']==4] = 0.2727
    return pred

In [22]:
pred = predict_by_age(test_data)
acc = accuracy_score(test_data['Survived'], pred > 0.5)
auc = roc_auc_score(test_data['Survived'], pred)
print('Accuracy = %.4f, AUC = %.4f' % (acc, auc))

Accuracy = 0.5341, AUC = 0.4819
